# Rock magnetic experiments data import

This notebook takes data that has been exported to MagIC format and unpacks it into its distinct tables. It also summarizes the specimens and the experiments that have been run on them.

## Import scientific python libraries

Run the cell below to import the functions needed for the notebook.

In [1]:
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

## Download and import data

### Download and unpack data from MagIC

Within the folder `./example_data/ECMB`, there is a MagIC contribution called `'ECMB 2018.TXT'`. Running the code cell below unpacks that MagIC contribution so that we can visualize and analyze the measurement data. We can then create a contribution object that has all of the MagIC tables.

In [2]:
# set the directory path (dir_path) to the data
dir_path = './example_data/ECMB'

ipmag.unpack_magic('ECMB 2018.TXT', 
                     dir_path = dir_path,
                     input_dir_path = dir_path)

contribution = cb.Contribution(dir_path)

working on:  'contribution\r'
1  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/contribution.txt
  data put in  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/contribution.txt
working on:  'specimens\r'
15  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/specimens.txt
  data put in  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/specimens.txt
working on:  'samples\r'
5  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/samples.txt
  data put in  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/samples.txt
working on:  'sites\r'
5  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/sites.txt
  data put in  /Users/unimos/0000_Github/RockmagPy-notebooks/example_data/ECMB/sites.txt
working on:  'locations\r'
10  records written to file  /Users/unimos/0000_Github/RockmagPy-notebooks/ex

### Inspect the measurements table

Within the MagIC contribution object, there is a measurements table that we can inspect.

In [3]:
measurements = contribution.tables['measurements'].df
measurements.dropna(axis=1, how='all').head() # see the first 5 measurements without empty columns

,experiment,instrument_codes,magn_mass,magn_r2_det,meas_field_dc,meas_temp,measurement,method_codes,quality,specimen,standard,timestamp,treat_dc_field
measurement name,,,,,,,,,,,,,
IRM-VSM3-LP-HYS-218845-1,IRM-VSM3-LP-HYS-218845,IRM-VSM3,0.1988,NaN,1.0010,300.0,IRM-VSM3-LP-HYS-218845-1,LP-HYS,g,NED1-5c,u,2020:01:06:10:14:00.00,NaN
IRM-VSM3-LP-HYS-218845-2,IRM-VSM3-LP-HYS-218845,IRM-VSM3,0.1986,NaN,0.9995,300.0,IRM-VSM3-LP-HYS-218845-2,LP-HYS,g,NED1-5c,u,2020:01:06:10:14:00.00,NaN
IRM-VSM3-LP-HYS-218845-3,IRM-VSM3-LP-HYS-218845,IRM-VSM3,0.1981,NaN,0.9966,300.0,IRM-VSM3-LP-HYS-218845-3,LP-HYS,g,NED1-5c,u,2020:01:06:10:14:00.00,NaN
IRM-VSM3-LP-HYS-218845-4,IRM-VSM3-LP-HYS-218845,IRM-VSM3,0.1974,NaN,0.9926,300.0,IRM-VSM3-LP-HYS-218845-4,LP-HYS,g,NED1-5c,u,2020:01:06:10:14:00.00,NaN
IRM-VSM3-LP-HYS-218845-5,IRM-VSM3-LP-HYS-218845,IRM-VSM3,0.1967,NaN,0.9880,300.0,IRM-VSM3-LP-HYS-218845-5,LP-HYS,g,NED1-5c,u,2020:01:06:10:14:00.00,NaN


### Get list of specimens and experiments

Now that we have imported the measurement data, we can look at the different experimental methods that have been applied to different specimens. The method codes correspond to the controlled vocabularies in the MagIC database.

| Method Code    | Definition |
| -------------- | ---------- |
| LP-BCR-BF      | Coercivity of remanence: Back field method | 
| LP-HYS:LP-HYS-M| Hysteresis loops |
| LP-CW-SIRM:LP-MC| Cycling between cooling and warming: Room temperature SIRM; Measured while cooling |
| LP-CW-SIRM:LP-MW| Cycling between cooling and warming: Room temperature SIRM; Measured while warming |
| LP-FC          | Field cooled: Remanent magnetization measured on warming |
| LP-ZFC         | Zero field cooled: Remanent magnetization measured on warming |


In [4]:
experiments = measurements.groupby(['specimen', 'method_codes']).size().reset_index().iloc[:, :2]
experiments

,specimen,method_codes
0,NED1-5c,LP-BCR-BF
1,NED1-5c,LP-HYS
2,NED18-2c,LP-BCR-BF
3,NED18-2c,LP-CW-SIRM:LP-MC
4,NED18-2c,LP-CW-SIRM:LP-MW
5,NED18-2c,LP-FC
6,NED18-2c,LP-HYS
7,NED18-2c,LP-ZFC
8,NED2-8c,LP-BCR-BF
9,NED2-8c,LP-CW-SIRM:LP-MC
